<a href="https://colab.research.google.com/github/theRubyPheonix/DRCN-/blob/main/SBDD_GCN_GAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep learning for structure-based drug discovery
## SBDD with deep learning-using GCN, GAT

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
cd '/content/drive/MyDrive/Colab Notebooks/GCN-Project'

/content/drive/MyDrive/Colab Notebooks/GCN-Project


In [20]:
# entities
import csv
import pandas as pd

entities_list = []
with open("entities.tsv", newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['entities','ids'])
    for row_val in reader:
        entities_list.append(row_val['entities'])
entity_df = pd.DataFrame(entities_list)
entity_df

,0
0,Gene::2157
1,Gene::5264
2,Gene::2158
3,Gene::3309
4,Gene::28912
...,...
97233,Gene::400359
97234,Gene::348751
97235,Gene::101928147
97236,Gene::23591


# Collecting COVID-19 related disease







At the very beginning we need to collect a list of disease of Corona-Virus(COV) in DRKG. We can easily use the Disease ID that DRKG uses for encoding the disease. Here we take all of the COV disease as target.

In [ ]:
COV_disease_list = [
'Disease::SARS-CoV2 E',
'Disease::SARS-CoV2 M',
'Disease::SARS-CoV2 N',
'Disease::SARS-CoV2 Spike',
'Disease::SARS-CoV2 nsp1',
'Disease::SARS-CoV2 nsp10',
'Disease::SARS-CoV2 nsp11',
'Disease::SARS-CoV2 nsp12',
'Disease::SARS-CoV2 nsp13',
'Disease::SARS-CoV2 nsp14',
'Disease::SARS-CoV2 nsp15',
'Disease::SARS-CoV2 nsp2',
'Disease::SARS-CoV2 nsp4',
'Disease::SARS-CoV2 nsp5',
'Disease::SARS-CoV2 nsp5_C145A',
'Disease::SARS-CoV2 nsp6',
'Disease::SARS-CoV2 nsp7',
'Disease::SARS-CoV2 nsp8',
'Disease::SARS-CoV2 nsp9',
'Disease::SARS-CoV2 orf10',
'Disease::SARS-CoV2 orf3a',
'Disease::SARS-CoV2 orf3b',
'Disease::SARS-CoV2 orf6',
'Disease::SARS-CoV2 orf7a',
'Disease::SARS-CoV2 orf8',
'Disease::SARS-CoV2 orf9b',
'Disease::SARS-CoV2 orf9c',
'Disease::MESH:D045169',
'Disease::MESH:D045473',
'Disease::MESH:D001351',
'Disease::MESH:D065207',
'Disease::MESH:D028941',
'Disease::MESH:D058957',
'Disease::MESH:D006517'
]

In [ ]:
!pip install dgl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 45.7 MB/s eta 0:00:00


In [ ]:
# GCN implementation with DGL
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [ ]:
# load the file 
import csv
drug_list=[]
with open('./infer_drug.tsv',newline='',encoding='utf-8') as csvfile:
  reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['drug','ids'])
  for row_val in reader:
    drug_list.append(row_val['drug'])

drug_list

['Compound::DB00605',
 'Compound::DB00983',
 'Compound::DB01240',
 'Compound::DB11755',
 'Compound::DB12184',
 'Compound::DB00404',
 'Compound::DB01223',
 'Compound::DB00572',
 'Compound::DB00669',
 'Compound::DB00494',
 'Compound::DB01092',
 'Compound::DB13800',
 'Compound::DB08893',
 'Compound::DB13678',
 'Compound::DB12274',
 'Compound::DB13375',
 'Compound::DB00215',
 'Compound::DB00268',
 'Compound::DB00115',
 'Compound::DB01172',
 'Compound::DB06781',
 'Compound::DB01118',
 'Compound::DB04695',
 'Compound::DB01180',
 'Compound::DB13312',
 'Compound::DB00344',
 'Compound::DB08909',
 'Compound::DB05325',
 'Compound::DB14541',
 'Compound::DB00514',
 'Compound::DB12693',
 'Compound::DB13402',
 'Compound::DB06774',
 'Compound::DB00982',
 'Compound::DB08897',
 'Compound::DB00415',
 'Compound::DB08986',
 'Compound::DB11675',
 'Compound::DB00026',
 'Compound::DB01611',
 'Compound::DB01559',
 'Compound::DB00363',
 'Compound::DB09232',
 'Compound::DB00091',
 'Compound::DB09143',
 'Compound

In [ ]:
treatment = ['Hetionet::CtD::Compound:Disease','GNBR::T::Compound:Disease']

# Get Pre-trained Model


In [ ]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(1, '/utils')
from utils import download_and_extract
download_and_extract()

ModuleNotFoundError: ignored